In [ ]:
#| code-fold: true
#| output: false

# setup code: install packages, API keys
library(tidyverse)
library(tidycensus)
census_api_key("4cf445b70eabd0b297a45e7f62f52c27ba3b5cae",
               install = TRUE, overwrite = TRUE)
Sys.setenv("CENSUS_KEY" = "4cf445b70eabd0b297a45e7f62f52c27ba3b5cae")
library(censusapi)
library(gtsummary)
library(knitr)
library(tigris)
library(maps)
library(ggplot2)

## Roadmap

A typical data science project:

<p align="center">

<img src="./data-science.png" width="80%"/>

</p>

### Learning objectives

In the next 1.5 days, we learn

-   the life cycle of a data science project

-   some R ecosystems (tidyverse, tidymodels) for Open Data Science

-   basic machine learning

-   policy evaluation using double machine learning

Dr. Roch Nianogo will lead the second part, from June 25 afternoon to June 26, with in-depth discussions of simulation modeling, causal inference, and linking data science and systems science.

### Course materials

During the course, you can

-   read the static tutorial pages, make comments, and ask questions; or

-   interactively run `qmd` codes in RStudio on [Posit Cloud](https://posit.cloud/spaces/522857/join?access_code=jp20j2u42A7Ygue-OICZ2U4S6IWZCnI5GFp0r_N9) (sign up for a free account); or

-   interactively run `ipynb` codes in Jupyter Notebook on [Binder](https://mybinder.org/v2/gh/NIH-R25-ModelersAndStoryTellers/binder-sandbox.git/main?urlpath=git-pull?repo=https://github.com/NIH-R25-ModelersAndStoryTellers/2024.git) (can be slow)

Adventurous ones can reproduce, improve, and generalize all the examples **on your own computer** by the following steps:

-   install [R](https://cran.r-project.org/), [RStudio](https://www.rstudio.com/products/rstudio/download/), and [Quarto](https://quarto.org/docs/get-started/)

-   git clone the [course repository](https://github.com/NIH-R25-ModelersAndStoryTellers/2024)

-   revise and render the qmd files

### Questions, please

Please feel free to ask questions and make comments. You can

-   use the "raise hand" feature (✋) in Zoom

-   type your questions in the Zoom chat (💬)

-   make comments or ask questions on tutorial pages (need to sign up an account on [hypothes.is](https://web.hypothes.is/))

## Data source

### Current Population Survey (CPS)

::: {style="text-align: center;"}
<img src="https://www.census.gov/etc.clientlibs/census/clientlibs/census-pattern-library/resources/images/USCENSUS_IDENTITY_SOLO_BLACK_1.5in_R_no_padding.svg" alt="Census Bureau" width="240"/>
:::

The [Current Population Survey (CPS)](https://www.census.gov/programs-surveys/cps.html), sponsored jointly by the U.S. Census Bureau and the U.S. Bureau of Labor Statistics (BLS), is the primary source of labor force statistics for the population of the United States.

The CPS is one of the oldest, largest, and most well-recognized surveys in the United States. It is immensely important, providing information on many of the things that define us as individuals and as a society – our work, our earnings, and our education.

In addition to being the primary source of monthly labor force statistics, the CPS is used to collect data for a variety of other studies that keep the nation informed of the economic and social well-being of its people. This is done by adding a set of supplemental questions to the monthly basic CPS questions. Supplemental inquiries vary month to month and cover a wide variety of topics such as child support, volunteerism, health insurance coverage, school enrollment, and **food security**. A listing and brief description of the CPS supplements are available [here](https://www.census.gov/programs-surveys/cps/about/supplemental-surveys.html).

### Food Security Supplement (CPS-FSS)

Take the **CPS Food Security Supplement December 2021 Public-Use Microdata File** as an example. The Food Security Supplement was completed for 30,343 interviewed households with 71,571 person records.

The microdata file includes data in four general categories:

-   Monthly labor force survey data (geographic, demographic, income, employment)
-   Food Security Supplement data (household food expenditures, use of food assistance programs, experiences and behaviors related to food security)
-   Food security status
-   Weighting variables

Food Security Supplement Questionnaire includes the following major sections:

-   Food Spending
-   Minimum Food Spending Needed
-   Food Assistance Program Participation
-   Food Sufficiency and Food Security
-   Ways of Avoiding or Ameliorating Food Deprivation

It is worth noting that beginning in 2015 and continuing through 2021, there were changes from previous years in how the Census Bureau processes some variables. Details can be found in the technical documentation, which can be found [here](https://www.census.gov/data/datasets/time-series/demo/cps/cps-supp_cps-repwgt/cps-food-security.html)

## Introduction to R

### Tidyverse

-   [tidyverse](https://www.tidyverse.org/) is a collection of R packages for data ingestion, wrangling, and visualization.

::: {style="text-align: center;"}
<img src="https://hbctraining.github.io/Intro-to-R/img/tidyverse_website.png" alt="Tidyverse"/>
:::

As it is difficult to change how fundamental base R structures/functions work, the Tidyverse suite of packages create and use data structures, functions and operators to make working with data more intuitive. The two most basic changes are in the use of pipes and tibbles.

-   The lead developer Hadley Wickham won the 2019 *COPSS Presidents’ Award* (the Nobel Prize of Statistics)

> for influential work in statistical computing, visualization, graphics, and data analysis; for developing and implementing an impressively comprehensive computational infrastructure for data analysis through R software; for making statistical thinking and computing accessible to large audience; and for enhancing an appreciation for the important role of statistics among data scientists.

#### Pipes

Stringing together commands in R can be quite daunting. Also, trying to understand code that has many nested functions can be confusing.

To make R code more human readable, the Tidyverse tools use the pipe, `%>%`, which was acquired from the **magrittr** package and comes installed automatically with Tidyverse. The pipe allows the output of a previous command to be used as input to another command instead of using nested functions.

In [ ]:
#| eval: true

# A single command
sqrt(83)

In [ ]:
# Base R method of running more than one command
round(sqrt(83), digit = 2)

In [ ]:
# Running more than one command with piping
sqrt(83) %>% round(digit = 2)

The pipe represents a much easier way of writing and deciphering R code, and we will be taking advantage of it for all future activities.

::: callout-tip
R 4.1.0 introduced a native pipe operator `|>`, which is mostly compatible with the pipe `%>%` offered by the tidyverse package magrittr. For some subtle differences, see this [post](https://www.tidyverse.org/blog/2023/04/base-vs-magrittr-pipe/) by Hadley Wickham.
:::

In [ ]:
#| eval: true

# R base pipe
sqrt(83) |> round(digit = 2)

#### Tibbles

A core component of the tidyverse is the tibble. Tibbles are a modern rework of the standard data.frame, with some internal improvements to make code more reliable. They are data frames, but do not follow all of the same rules. For example, tibbles can have column names that are not normally allowed, such as numbers/symbols.

The main differences between tibbles and data.frames relate to printing and subsetting.

-   `iris` is a data frame available in base R

::: {style="text-align: center;"}
<img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*ZK9_HrpP_lhSzTq9xVJUQw.png" alt="iris" width="500"/>
:::

In [ ]:
#| eval: true

# By default, R displays ALL rows of a regular data frame!
iris

-   Convert a regular data frame to tibble, which by default only displays the first 10 rows of data.

In [ ]:
#| eval: true

# Convert iris to a tibble
iris_tb <- as_tibble(iris)
iris_tb

In [ ]:
#| eval: true

# If subsetting a single column from a data.frame, R will output a vector
iris[, "Sepal.Length"]

In [ ]:
#| eval: true

# If subsetting a single column from a tibble, R will output a tibble
iris_tb[, "Sepal.Length"]

Also note that if you use piping to subset a tibble, then the notation is slightly different, requiring a placeholder `.` prior to the `[ ]` or `$`.

In [ ]:
#| eval: false

# Return a vector
iris_tb$Sepal.Length
iris_tb[["Sepal.Length"]]
iris_tb[[1]]

# Return a tibble
iris_tb[, "Sepal.Length"]
iris_tb[, 1]

# Use piping
iris_tb %>% .$Sepal.Length
iris_tb %>% .[, "Sepal.Length"]

#### dplyr

The most useful tool in the tidyverse is dplyr. It’s a Swiss-army knife for data wrangling. dplyr has many handy functions that we recommend incorporating into your analysis.

-   Operations on rows:
    -   `arrange()` changes the ordering of the rows.
    -   `filter()` picks cases based on their values.
    -   `distinct()` removes duplicate entries.
    -   `slice_*()` selects rows by position.
-   Operations on columns:
    -   `select()` extracts columns and returns a tibble.
    -   `mutate()` adds new variables that are functions of existing variables.
    -   `rename()` easily changes the name of a column(s).
    -   `pull()` extracts a single column as a vector.
-   Grouped operations:
    -   `group_by()` aggregates data by one or more variables.\
    -   `summarise()` reduces multiple values down to a single summary.
-   `_join()` functions that merge two data frames together, including `inner_join()`, `left_join()`, `right_join()`, and `full_join()`.

The [Posit dplyr cheatsheet](https://rstudio.github.io/cheatsheets/data-transformation.pdf) is extremely intuitive and helpful.

Some examples of using dplyr functions.

-   Filter observations with `Sepal.Length` greater than 5.0, arrange the data by `Sepal.Length` in descending, and create a new column `Sepal.Length_2` that is the square of `Sepal.Length`.

In [ ]:
#| eval: true

iris_tb |>
  filter(Sepal.Length > 5.0) |>
  arrange(desc(Sepal.Length)) |>
  mutate(Sepal.Length_2 = Sepal.Length^2) |>
  print()

-   Select columns `Species`, and find the distinct values of `Species`

In [ ]:
#| eval: true

iris_tb |>
  select(Species) |>
  distinct()

-   Count the number of rows in each species

In [ ]:
#| eval: true

iris_tb |>
  group_by(Species) |> 
  summarize(n = n())

In [ ]:
# Shortcut for group_by() |> summarize(n = n())
iris_tb |>
  count(Species)

-   Calculate the mean of `Sepal.Length` for each `Species`

In [ ]:
#| eval: true

iris_tb |>
  group_by(Species) |>
  summarize(mean_Sepal_Length = mean(Sepal.Length))

-   Find the observation with the maximum `Sepal.Length` for each `Species`

In [ ]:
#| eval: true

iris_tb |>
  group_by(Species) |>
  slice_max(Sepal.Length)

#### Combine variables (columns)

-   Demo tables

In [ ]:
#| eval: true

(x <- tribble(
  ~key, ~val_x,
  1, "x1",
  2, "x2",
  3, "x3"
))

In [ ]:
#| eval: true

(y <- tribble(
  ~key, ~val_y,
  1, "y1",
  2, "y2",
  4, "y3"
))

-   An **inner join** matches pairs of observations whenever their keys are equal:

<p align="center">

<img src="./join-inner.png" height="150"/>

</p>

In [ ]:
#| eval: true

inner_join(x, y, by = "key")

-   An **outer join** keeps observations that appear in at least one of the tables.

-   Three types of outer joins: **left join**, **right join**, and **full join**.

<p align="center">

<img src="./join-outer.png" width="50%"/>

</p>

-   A **left join** keeps all observations in `x`.

In [ ]:
#| eval: true

left_join(x, y, by = "key")

-   A **right join** keeps all observations in `y`.

In [ ]:
#| eval: true

right_join(x, y, by = "key")

-   A **full join** keeps all observations in `x` and `y`.

In [ ]:
#| eval: true

full_join(x, y, by = "key")

-   One table has duplicate keys.

<p align="center">

<img src="./join-one-to-many.png" height="200"/>

</p>

In [ ]:
#| eval: true

x <- tribble(
  ~key, ~val_x,
  1, "x1",
  2, "x2",
  2, "x3",
  1, "x4"
)
y <- tribble(
  ~key, ~val_y,
  1, "y1",
  2, "y2"
)
left_join(x, y, by = "key")

-   Both tables have duplicate keys. You get all possible combinations, the Cartesian product:

<p align="center">

<img src="./join-many-to-many.png" height="250"/>

</p>

In [ ]:
#| eval: true

x <- tribble(
  ~key, ~val_x,
  1, "x1",
  2, "x2",
  2, "x3",
  3, "x4"
)
y <- tribble(
  ~key, ~val_y,
  1, "y1",
  2, "y2",
  2, "y3",
  3, "y4"
)

left_join(x, y, by = "key")

#### Combine cases (rows)

-   `semi_join(x, y)` keeps the rows in `x` that have a match in `y`.

<p align="center">

<img src="./join-semi.png" height="200"/>

</p>

In [ ]:
#| eval: true

x <- tribble(
  ~key, ~val_x,
  1, "x1",
  2, "x2",
  3, "x3"
)

y <- tribble(
  ~key, ~val_y,
  1, "y1",
  2, "y2",
  4, "y3"
)

semi_join(x, y, by = "key")

-   `anti_join(x, y)` keeps the rows that don’t have a match.

<p align="center">

<img src="./join-anti.png" height="200"/>

</p>

In [ ]:
#| eval: true

anti_join(x, y, by = "key")

## Ingest CPS-FSS Data

Many datasets such as the Decennial Census since 2000 and the American Community Survey (ACS) are available through the US Census Bureau’s APIs and in turn accessible with [`tidycensus`](https://walker-data.com/tidycensus/) and related tools. However, the CPS-FSS data is not available through the `tidycensus` package, which only focuses on core datasets. Other R packages provide support for the wide range of datasets available from the Census Bureau and other government agencies. `censusapi` package, for example, allows for programmatic access to all US Census Bureau APIs.

### Sign up for a Census API key

Sign up for a Census API key at <https://api.census.gov/data/key_signup.html> and install the key.

```         
# Set the Census API key in the environment
Sys.setenv("CENSUS_KEY" = "PUT YOUR KEY HERE")
```

### censusapi package

[censusapi](https://github.com/hrecht/censusapi) is a lightweight package to get data from the U.S. It uses the same Census API key as `tidycensus`, though references it with the R environment variable `CENSUS_KEY`. If this environment variable is set in a user’s `.Renviron` file, functions in `censusapi` will pick up the key without having to supply it directly.

-   `getCensus()`: Retrieve Census data from a given API.
-   `listCensusApis()`: Get useful dataset metadata on all available APIs as a data frame.
-   `listCensusMetadata()`: Get information about a specific API as a data frame.
-   `makeVarlist()`: Use variable metadata to find variables containing a given string.

`censusapi`’s core function is `getCensus()`, which translates R code to Census API queries. The `name` argument references the API name; the `censusapi` documentation or the function `listCensusApis()` helps you understand how to format this.

To see a current table of every available endpoint, run `listCensusApis()`:

In [ ]:
#| eval: true

censusapi::listCensusApis() |> 
  # convert data.frame to tibble
  as_tibble() |>
  select(title, name, vintage, type, temporal, url)

`listCensusApis()` returns a dataframe that includes: title, description, name, vintage, url, dataset type, and other useful fields. Search for the dataset you are interested in.

Now we are interested in Food Security Supplement. We can search for the keyword "Food Security" in the column `title` of the table above, and see which year of data is available.

In [ ]:
#| eval: true

censusapi::listCensusApis() |>
  filter(str_detect(title, "Food Security"), vintage >= 2019) |>
  select(title, name, vintage, type, description)

The example below makes a request to API for the CPS Food Security Supplement December 2021 Public-Use Microdata File. The `name` argument is set to "cps/foodsec/dec" and the `vintage` argument is set to 2021. If we are interested in Household Food Security Scale, then for the `vars` argument, we should include the following terms:

-   HRHHID: Household ID
-   HRHHID2: Household ID (Part 2)
-   PERRP: Relationship to Reference Person
-   GESTFIPS: State FIPS Code
-   GTCO: County Code
-   HRFS12M1: Summary Food Security Status, 12-Month
    -   1 = Food Secure High or Marginal Food Security
    -   2 = Low Food Security
    -   3 = Very Low Food Security
    -   -1 = Not in Universe (In this variable, not interviewed)
    -   -9 = No Response
-   HHSUPWGT: Household Supplemental Weight

In [ ]:
#| eval: true

fss_21_status <- censusapi::getCensus(
  name = "cps/foodsec/dec",
  vintage = 2021,
  vars = c("HRHHID", "HRHHID2", "PERRP",
           "GESTFIPS", "GTCO", "HRFS12M1",
           "HHSUPWGT")
  ) |>
  as_tibble()

fss_21_status

Notice that some columns are not in the format we want. For example, **HRFS12M1** (Food Security Status) should be a categorical variable, and we want to convert it to a factor with meaningful labels. Also, **HHSUPWGT** (Household Supplemental Weight) was ingested as a character variable, and we want to convert it to a numeric variable.

In [ ]:
#| eval: true

fss_21_status |>
  filter(HRFS12M1 != "-1") |>
  mutate(HRFS12M1 = factor(HRFS12M1,
                           levels = c("1", "2", "3", "-9"),
                           labels = c("Food Security",
                                      "Low Food Security",
                                      "Very Low Food Security",
                                      "No Response")),
         HHSUPWGT = as.numeric(HHSUPWGT),
         PERRP = as.numeric(PERRP),
         HRHHID2 = as.character(HRHHID2))

In addition to checking the document and encoding labels manually, `listCensusMetadata()` offers a way to get the value labels of specific variables. This can be useful for understanding the meaning of variables and their values.

In [ ]:
#| eval: true

HRFS12M1_lb <- censusapi::listCensusMetadata(name = "cps/foodsec/dec",
                                             vintage = 2021,
                                             type = "values",
                                             variable = "HRFS12M1")

HRFS12M1_lb

Therefore, we can use the following code to convert the variable `HRFS12M1` to a factor with meaningful labels.

In [ ]:
#| eval: true

fss_21_status <- fss_21_status |>
  filter(HRFS12M1 != "-1") |>
  mutate(HRFS12M1 = factor(HRFS12M1,
                           levels = HRFS12M1_lb$code,
                           labels = HRFS12M1_lb$label),
         HHSUPWGT = as.numeric(HHSUPWGT),
         PERRP = as.numeric(PERRP),
         HRHHID2 = as.character(HRHHID2))
  
fss_21_status

## Constructing household characteristics from person records

To compute some household characteristics (such as household size, presence of children, or presence of elderly members), it is necessary to identify the records of all persons in the same household. Households within the December CPS-FSS are uniquely and completely identified by two household identifiers in combination, **HRHHID** and **HRHHID**. Characteristics of the household reference person can be assigned from the person record with **PERRP** 40 or 41, which will always be the record with the lowest-numbered PERRP in the household.

**HRFS12M1** (Summary Food Security Status, 12-Month) is one of the household characteristics. This is the variable used for most food security statistics in USDA’s annual food security report series. In order to compute the prevalence of food insecurity, we need to aggregate the food security status of all persons in the same household. `tbl_summary()` in the `gtsummary` package can generate beautiful summary tables.

In [ ]:
#| eval: true

fss_21_status <- fss_21_status |>
  # Filter observations with PERRP 40 or 41 (reference person in the household) 
  filter(PERRP %in% c(40, 41)) |>
  # Arrange by PERRP
  arrange(PERRP) |>
  # Keep the record with the lowest-numbered PERRP in the household
  slice_head(by = c(HRHHID, HRHHID2), n = 1)

fss_21_status

In [ ]:
# Display number and proportion of households in different food security status
fss_21_status |>
  select(HRFS12M1) |>
  gtsummary::tbl_summary()

Notice that there are 27,357 + 1,844 + 1,093 + 49 = 30,343 households who attended the interview for Food Security Supplement in 2021, which matches the number in the technical documentation.

The CPS is a complex probability sample, and interviewed households, as well as persons in those households, are assigned weights so that the full interviewed sample represents the total national non-institutionalized civilian population. Initial weights are assigned based on probability of selection into the sample, and weights are then adjusted iteratively to match population controls for selected demographic characteristics at State and national levels. There are two sets of household and person weights in this data file: (1) labor force survey weights, and (2) Food Security Supplement weights.

We can use `makeVarlist()` function in `censusapi` package to get the list of variables in the dataset. In addition to the description and the type of each variable, we can also check column **suggested_weight** to see which weight should be used for the analysis.

In [ ]:
#| eval: true

censusapi::makeVarlist(
  name = "cps/foodsec/dec",
  vintage = 2021,
  find = "",
  output = "dataframe"
  ) |>
  as_tibble() |>
  filter(name %in% c("HRHHID", "HRHHID2", "PERRP",
                     "GESTFIPS", "HRFS12M1", "HHSUPWGT")) |>
  select(-c(concept, group, predicateOnly))

Now we are interested in how much percentage of households with low food security and very low food security in 2021.

-   Identify the variables needed:

    HRFS12M1: Summary Food Security Status, 12-Month

    HRFS12M1 = 2 OR HRFS12M1 = 3 (Low Food Security OR Very Low Food Security)

-   Identify the statistical weight needed:

    HHSUPWGT: Household Weight for Supplement Household

-   Perform the calculation:

    The rate is given by

    $$
    \text{Rate} = \frac{\sum_i w_i \cdot L_i}{\sum_i w_i \cdot H_i} \times 100\%
    $$

    where $L_i$ is an indicator variable that equals 1 if household $i$ is in low food security or very low food security, and 0 otherwise; $H_i$ is an indicator variable that equals 1 if household $i$ attended the interview for Food Security Supplement and had a response to this question, and 0 otherwise; and $w_i$ is the weight of household $i$.

In [ ]:
#| eval: true

fss_21_status <- fss_21_status |>
  mutate(HRFS12M1_low = ifelse(HRFS12M1 %in% c("Low Food Security",
                                               "Very Low Food Security"),
                               1, 0) * HHSUPWGT,
         HRFS12M1_res = ifelse(HRFS12M1 == "No Response", 0, 1) * HHSUPWGT
  )

low_food_security_rate <-
  sum(fss_21_status$HRFS12M1_low) / sum(fss_21_status$HRFS12M1_res) * 100

low_food_security_rate

The percentage of households with low food security or very low food security in 2021 is 10.23%. We can verify this result by comparing this number to the Economic Research Report [Household Food Security in the United States in 2021](https://www.ers.usda.gov/webdocs/publications/104656/err-309.pdf?v=7855.7) Figure 1.

Since we have already computed the rate of low food security in 2021. We can further investigate the rate of low food security in each state.

We first need to match the FIPS code to the state names using `fips_codes()` function in `tigris` package.

In [ ]:
#| eval: true

# Get the state names
state_names <- tigris::fips_codes |>
  select(state, state_code, state_name) |>
  distinct() |>
  as_tibble()

state_names

Now we can compute the rate of low food security in each state.

In [ ]:
#| eval: true

fss_21_status_state <- fss_21_status |>
  
  # Group by state FIPS code and calculate the rate by state
  group_by(GESTFIPS) |>
  summarize(HRFS12M1_low = sum(HRFS12M1_low),
            HRFS12M1_res = sum(HRFS12M1_res)) |>
  
  # Notice that state_code is a two digit number
  # But in fss_21_status, it is single digit if the state code is less than 10
  mutate(GESTFIPS = ifelse(as.numeric(GESTFIPS) < 10,
                           paste0("0", GESTFIPS),
                           as.character(GESTFIPS))) |>
  
  # Left join with state names
  left_join(state_names, by = c("GESTFIPS" = "state_code")) |>
  mutate(low_food_security_rate = HRFS12M1_low / HRFS12M1_res * 100,
         state_name = str_to_lower(state_name))

Then we can visualize the rate of low food security in each state.

In [ ]:
#| eval: true

us_states <- ggplot2::map_data("state")

us_states <- merge(us_states, fss_21_status_state, by.x = "region",
                   by.y = "state_name", all.x = TRUE)

ggplot(us_states, aes(x = long, y = lat, group = group,
                      fill = low_food_security_rate)) +
  geom_polygon(color = "black") +
  scale_fill_gradient(low = "lightblue", high = "darkblue",
                      name = "Households in Low Food Security (%)") +
  labs(title = "Percentage of Households in Low Food Security by State") +
  theme_minimal() +
  theme(panel.grid = element_blank(),
        axis.text = element_blank(),
        axis.title = element_blank(),
        legend.position = "bottom") +
  coord_fixed(ratio = 1.5)